<a href="https://colab.research.google.com/github/TejaNaveenChintapalli/RL_assignment1/blob/main/SARSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class SARSA_():
  def __init__(self, Environment, Learning_Rate = 0.01, n_episodes = 500, Epsilon = 1.0, Discount_Factor = 0.3, Epsilon_Decay = 0.01):
    self.alpha = Learning_Rate
    self.n_episodes = n_episodes
    self.epsilon = Epsilon
    self.gamma = Discount_Factor
    self.epsilon_decay = Epsilon_Decay
    self.env = Environment
    self.Q_Table = np.random.random((self.env.n_states, self.env.n_actions))
    self.Q_Table[self.env.n_states - 1,:] = 0
    self.state_map = []
    states_count = 0
    for i in range(self.env.n_rows):
      self.state_map.append([])
      for j in range(self.env.n_columns):
        self.state_map[i].append(states_count)
        states_count += 1
    self.games_reward = []
    self.decay_growth = []
    self.evaluation_results = []
    pass
  def plot_graph(self, data=None, x_label="", y_label="", title=""):
    '''Plotting graph with given data'''
    if data is None:
      print("There isn't any data to plot")
      return 
    print("\n")
    plt.plot(data)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()
    pass
  def greedy(self, cur_state):
    ''' Greedy Policy'''
    return np.argmax(self.Q_Table[cur_state])
  
  def epsilon_greedy(self, cur_state):
    '''Epsilon greedy policy'''
    if np.random.uniform(0,1) < self.epsilon:
        return self.env.action_space.sample()
    else:
        return self.greedy(cur_state)
  
  def train_Model(self):
    for episode in range(self.n_episodes):
      self.env.reset()
      state = self.state_map[self.env.agent_pos[0]][self.env.agent_pos[1]]
      done = False
      self.epsilon = max(0.01, self.epsilon - (self.epsilon * self.epsilon_decay))
      
      action = self.epsilon_greedy(state) 
      # Running actions until environment reaches final goal or exceeds max timesteps
      while not done:
        observation, cur_reward, done, _ = self.env.step(action)
        next_state = self.state_map[self.env.agent_pos[0]][self.env.agent_pos[1]]
        next_action = self.epsilon_greedy(next_state)
        prev_q = tuple([state, action])
        cur_q = tuple([next_state, next_action])
        
        G = cur_reward + self.gamma * self.Q_Table[cur_q]
        self.Q_Table[prev_q] = self.Q_Table[prev_q] + self.alpha * (G - self.Q_Table[prev_q])

        state = next_state
        action = next_action
        if done:
          self.games_reward.append(self.env.total_reward)
          self.decay_growth.append(self.epsilon)
  def evaluate_model(self, test_n_episodes = 10):
    output_grid = widgets.Grid(1,1)
    for ep in range(test_n_episodes):
      self.env.reset()
      state = self.state_map[self.env.agent_pos[0]][self.env.agent_pos[1]]
      done = False
      tot_rew = 0
      with output_grid.output_to(0,0):
        self.env.render()
        time.sleep(0.2)
        output_grid.clear_cell()
      while not done:
        action = np.argmax(self.Q_Table[state])
        observation, rew, done, _ = self.env.step(action) 
        state = self.state_map[self.env.agent_pos[0]][self.env.agent_pos[1]]

        tot_rew += rew
        with output_grid.output_to(0,0):
          self.env.render()
          time.sleep(0.2)
          output_grid.clear_cell()
        if done:
          self.evaluation_results.append(tot_rew)
    pass
